# Project Proposal
Group 60 - Danyal, Ruth, Thomas, Paul

In [2]:
##Title: Predicting Heart disease based on data

## Introduction (Ruth)

## Preliminary exploratory data analysis (Thomas)

## Methods (Paul)

## Expected outcomes and significance (Danyal)